In [ ]:
!pip install das

In [1]:
import numpy as np
from pprint import pprint
import scipy.io.wavfile

import pandas as pd
import os
import glob

import das.predict

In [2]:
def das_predict(INPUT,model_name,out):
    samplerate, x = scipy.io.wavfile.read(INPUT)
    x = np.atleast_2d(x).T
    events, segments, class_probabilities, class_names = das.predict.predict(x, 
                                                               model_save_name=model_name,
                                                               verbose=2,
                                                               segment_minlen=0.02,
                                                               segment_fillgap=0.02)
    arr1 = segments['onsets_seconds']
    arr2 = segments['offsets_seconds']
    arr3 = [[segments['sequence'][i]] for i in range(len(segments['sequence']))]
    df = pd.DataFrame(np.array([np.array(arr3).flatten(),arr1,arr2]).T,columns=['name','onsets_seconds','offsets_seconds'])
    df.columns = ['name','start_seconds','stop_seconds']
    df_pred = df.loc[df.name!='noise']
    path,fname = os.path.split(INPUT)
    file,ext = os.path.splitext(fname)
    del df,x,samplerate ## this is to clear memory. 
    df_pred.to_csv(os.path.join(out,file+'_annotations.csv'),index=False)

In [7]:
home_path = '/Volumes/RENATA_X/Vallentinlab'
birdfolder = 'bird7159'
channel = 'chan6'

model_name = '/Volumes/RENATA_X/Vallentinlab/models/'+birdfolder+'_'+channel+'.res/20220629_103442'
infiles = glob.glob(os.path.join(home_path,birdfolder+'_sorted',channel,'dph*_filtered_denoised_sampled.wav'))
INPUT = infiles[1]
samplerate, x = scipy.io.wavfile.read(INPUT)
x = np.atleast_2d(x).T
events, segments, class_probabilities, class_names = das.predict.predict(x, 
                                                           model_save_name=model_name,
                                                           verbose=2,
                                                           segment_minlen=0.02,
                                                           segment_fillgap=0.02)

In [ ]:
home_path = '/Volumes/RENATA_X/Vallentinlab'
birdfolder = 'bird7159'
channel = 'chan6'

model_name = '/Volumes/RENATA_X/Vallentinlab/models/'+birdfolder+'_'+channel+'.res/20220629_103442'
infiles = glob.glob(os.path.join(home_path,birdfolder+'_sorted',channel,'dph*_filtered_denoised_sampled.wav'))
for infile in infiles:
    das_predict(infile,model_name,os.path.join(home_path,birdfolder+'_sorted',channel))

DAS requires [T, channels], but single-channel wave files are loaded with shape [T,] (data shape is (7200000,)).


2022-09-18 22:52:44.869949: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/yirongx/opt/miniconda3/envs/das/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


DAS requires [T, channels], but single-channel wave files are loaded with shape [T,] (data shape is (7200000,)).


/Users/yirongx/opt/miniconda3/envs/das/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


DAS requires [T, channels], but single-channel wave files are loaded with shape [T,] (data shape is (7200000,)).


...

/Users/yirongx/opt/miniconda3/envs/das/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


DAS requires [T, channels], but single-channel wave files are loaded with shape [T,] (data shape is (7200000,)).


/Users/yirongx/opt/miniconda3/envs/das/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


DAS requires [T, channels], but single-channel wave files are loaded with shape [T,] (data shape is (7200000,)).


/Users/yirongx/opt/miniconda3/envs/das/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
